In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt
from __future__ import print_function
%matplotlib inline

/Users/andyxie/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# MODEL

### Constants

In [4]:
LEARNING_RATE = 1e-4 # Lambda???
N_INPUTS = 784
N_OUTPUTS = 10
N_UNITS = 1024
N_STEPS = 1000
BATCH_SIZE = 100

### Combine Keras and TensorFlow

In [5]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


### Placeholder

In [6]:
X = tf.placeholder(tf.float32, shape=(None, N_INPUTS))
y = tf.placeholder(tf.float32, shape=(None, N_OUTPUTS))

### Model

In [7]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(N_UNITS, activation='relu', input_shape = (None, N_INPUTS)))
model.add(Dense(N_UNITS, activation='relu'))
model.add(Dense(N_OUTPUTS, activation='softmax'))
y_ = model(X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, None, 1024)        803840    
_________________________________________________________________
dense_2 (Dense)              (None, None, 1024)        1049600   
_________________________________________________________________
dense_3 (Dense)              (None, None, 10)          10250     
Total params: 1,863,690
Trainable params: 1,863,690
Non-trainable params: 0
_________________________________________________________________


### Loss Function

In [8]:
from keras.objectives import categorical_crossentropy
with tf.name_scope("Loss"):
    loss = tf.reduce_mean(categorical_crossentropy(y, y_))
    tf.summary.scalar("Loss", loss)

### Optimizer

In [9]:
step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

### Metrics

In [10]:
from keras.metrics import categorical_accuracy
metrics = tf.reduce_mean(categorical_accuracy(y, y_))


# Session

In [11]:
# Initialize all variables
init = tf.global_variables_initializer()
sess.run(init)

# Log
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/", sess.graph)

In [14]:
with sess.as_default():
    for i in range(N_STEPS):
        batch = mnist.train.next_batch(50)
        feed_dict = {X:batch[0], y:batch[1]}
        sess.run(step, feed_dict = feed_dict)
        if i % 50 == 0:
            # Log
            result = sess.run(merged, feed_dict = feed_dict)
            writer.add_summary(result, i)
            